In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, EarlyStoppingCallback, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoModelForPreTraining
from collections import OrderedDict
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
path = "/content/drive/MyDrive/Colab_Notebooks/cgpt_classification/Data/"

#r"C:\Saeid\Prj100\SA_45_DSS_embrace"
train_file = "seen_new.xlsx"
test_file = "unseen_new.xlsx"
train_path = os.path.join(path, train_file)
test_path = os.path.join(path, test_file)
#cgpt_path = r"C:\Saeid\Prj100\SA_45_DSS_embrace\models\climateGPT2_ready_RL\pytorch_model.bin" # the path to bin file of cgpt((this is ckeygpt do we want cgpt or this?))
#output_path = r"C:\Saeid\Prj100\SA_45_DSS_embrace\relevancy_cgpt_save"

cgpt_path = "/content/drive/MyDrive/Colab_Notebooks/cgpt_classification/models/climateGPT2_ready_RL/pytorch_model.bin"
output_path = "/content/drive/MyDrive/Colab_Notebooks/cgpt_classification/relevancy_cgpt_save/"

In [ ]:
#set_epoch = 100
set_epoch = 2
train_data = pd.read_excel(train_path)
print(train_data.head())

test_data = pd.read_excel(test_path)
print(test_data.head())

train_data.describe()

test_data.describe()

ready_to_train = train_data[["Abstract"]]
print(ready_to_train.head())

train, test = train_test_split(train_data[["Abstract", "Relevant"]], test_size=0.33, random_state=42)

print(train.head())
print(test.head())
class relevancyDataset(Dataset):

    def __init__(self, data, tokenizer):
        x = data["Abstract"].tolist()
        label = data["Relevant"].tolist()

        self.tokenizer = tokenizer
        self.x = x
        self.label = label

    # ---------------------------------------------#

    def __len__(self):
        return len(self.label)

    # ---------------------------------------------#

    def __getitem__(self, i):
        encodings_dict = tokenizer(self.x[i],
                                   truncation=True,
                                   max_length=1500,
                                   padding="max_length")

        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']

        return {'label':torch.tensor(self.label[i]),
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask)}


                           DOI  \
0   10.1016/j.nedt.2021.105144   
1   10.1007/s11422-021-10080-6   
2     10.1016/j.pt.2010.06.009   
3            10.1111/jnu.12326   
4  10.1016/j.jsurg.2020.11.001   

                                               Title  \
0  A mixed method study on global warming, climat...   
1  Global challenges need attention now: educatin...   
2  Soil-transmitted helminthiases: implications o...   
3  Climate Change, Climate Justice, and Environme...   
4  The Role of Preoperative Briefing and Postoper...   

                                            Abstract  \
0  AIM: This study aims to evaluate the knowledge...   
1  This Editorial sets the stage for 18 papers on...   
2  Soil-transmitted helminthiases (STHs) collecti...   
3  PURPOSE: Climate change is an emerging challen...   
4  OBJECTIVE: To study the impact of a new preope...   

                            Source title  PubYear  \
0                  Nurse Education Today     2021   
1  Cultural Stu

In [ ]:
base_model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

tt = GPT2Tokenizer.from_pretrained(base_model)
tt.pad_token = tt.eos_token

train_data_set = relevancyDataset(train, tokenizer)
test_data_set = relevancyDataset(test, tokenizer)


model_final = AutoModelForSequenceClassification.from_pretrained(output_path)
model_final.config.pad_token_id = model_final.config.eos_token_id
xd = []
dx = []
for k, i in enumerate(test_data["Abstract"]):
    print(str(k)+"/"+str(len(test_data["Abstract"])))
    inputs = tokenizer(i,truncation=True,max_length=750,padding="max_length", return_tensors='pt')
    logits = model_final(**inputs).logits

    softmax= torch.nn.Softmax()

    predicted_class_id = logits.argmax().item()
    prcnt = round(softmax(logits).max().item()*100, 2)


    xd.append(predicted_class_id)
    dx.append(prcnt)
req = pd.DataFrame({"Abstract":test_data["Abstract"], "Relevant": xd, "DOI":test_data["DOI"], "Title":test_data["Title"], "Source title":test_data["Source title"],"PubYear":test_data["PubYear"],"Authors":test_data["Authors"],"Times cited":test_data["Times cited"],"Probability":dx})
#print(req.head())

0/16346


<ipython-input-15-1365f7b3de18>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prcnt = round(softmax(logits).max().item()*100, 2)


Streaming output truncated to the last 5000 lines.
11346/16346
11347/16346
11348/16346
11349/16346
11350/16346
11351/16346
11352/16346
11353/16346
11354/16346
11355/16346
11356/16346
11357/16346
11358/16346
11359/16346
11360/16346
11361/16346
11362/16346
11363/16346
11364/16346
11365/16346
11366/16346
11367/16346
11368/16346
11369/16346
11370/16346
11371/16346
11372/16346
11373/16346
11374/16346
11375/16346
11376/16346
11377/16346
11378/16346
11379/16346
11380/16346
11381/16346
11382/16346
11383/16346
11384/16346
11385/16346
11386/16346
11387/16346
11388/16346
11389/16346
11390/16346
11391/16346
11392/16346
11393/16346
11394/16346
11395/16346
11396/16346
11397/16346
11398/16346
11399/16346
11400/16346
11401/16346
11402/16346
11403/16346
11404/16346
11405/16346
11406/16346
11407/16346
11408/16346
11409/16346
11410/16346
11411/16346
11412/16346
11413/16346
11414/16346
11415/16346
11416/16346
11417/16346
11418/16346
11419/16346
11420/16346
11421/16346
11422/16346
11423/16346
11424/16346
1

In [ ]:
ff = []
for k, i in enumerate(train_data["Abstract"]):
    print(str(k)+"/"+str(len(train_data["Abstract"])))
    ff.append(100.0)
qer = pd.DataFrame({"Abstract":train_data["Abstract"], "Relevant": train_data["Relevant"], "DOI":train_data["DOI"], "Title":train_data["Title"], "Source title":train_data["Source title"],"PubYear":train_data["PubYear"],"Authors":train_data["Authors"],"Times cited":train_data["Times cited"],"Probability":ff})

print("END!!")
print(qer.head())

result = pd.concat([qer, req])
print(result.head())
rls = shuffle(result)
print(rls.head())
rls.to_excel("/content/drive/MyDrive/Colab_Notebooks/cgpt_classification/Data/output_relevant_unseen_seen_merge_shuffle_August2023.xlsx")

0/1766
1/1766
2/1766
3/1766
4/1766
5/1766
6/1766
7/1766
8/1766
9/1766
10/1766
11/1766
12/1766
13/1766
14/1766
15/1766
16/1766
17/1766
18/1766
19/1766
20/1766
21/1766
22/1766
23/1766
24/1766
25/1766
26/1766
27/1766
28/1766
29/1766
30/1766
31/1766
32/1766
33/1766
34/1766
35/1766
36/1766
37/1766
38/1766
39/1766
40/1766
41/1766
42/1766
43/1766
44/1766
45/1766
46/1766
47/1766
48/1766
49/1766
50/1766
51/1766
52/1766
53/1766
54/1766
55/1766
56/1766
57/1766
58/1766
59/1766
60/1766
61/1766
62/1766
63/1766
64/1766
65/1766
66/1766
67/1766
68/1766
69/1766
70/1766
71/1766
72/1766
73/1766
74/1766
75/1766
76/1766
77/1766
78/1766
79/1766
80/1766
81/1766
82/1766
83/1766
84/1766
85/1766
86/1766
87/1766
88/1766
89/1766
90/1766
91/1766
92/1766
93/1766
94/1766
95/1766
96/1766
97/1766
98/1766
99/1766
100/1766
101/1766
102/1766
103/1766
104/1766
105/1766
106/1766
107/1766
108/1766
109/1766
110/1766
111/1766
112/1766
113/1766
114/1766
115/1766
116/1766
117/1766
118/1766
119/1766
120/1766
121/1766
122/1766
123